## Criando uma Rede Neural com PyTorch

Usando o dataset da Iris Flowers do Scikit Learn (que está no formato numpy), nós construímos uma rede neural com PyTorch para prever a espécie da flor. Começamos carregando o dataset e visualizando as propriedades do mesmo.

In [23]:
from sklearn.datasets import load_iris
iris_data = load_iris()
features, pre_labels = iris_data.data, iris_data.target

In [24]:
features, pre_labels

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
        [5

In [3]:
print(type(features))
print(type(pre_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


## Pré-processamento das labels em um array

Nós vamos converter as classes em um array. Por exemplo, a classe[0] será [1,0,0]. Depois fazemos um "casting" pra array de numpy para melhor computação e memória.

In [25]:
labels = []
for num in range(len(pre_labels)):
    if pre_labels[num] == 0:
        labels.append([1, 0, 0])
    if pre_labels[num] == 1:
        labels.append([0, 1, 0])
    if pre_labels[num] == 2:
        labels.append([0, 0, 1])

In [26]:
import numpy as np
labels = np.array(labels, dtype = int)

## Dividir o dataset em treinamento e teste

Nós importamos uma função do sci-kit learn para dividir o dataset em treinamento e teste em 70% - 30%.

In [27]:
from sklearn.model_selection import train_test_split
feature_train, feature_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.30, random_state=0)

In [28]:
print(len(feature_train))
print(len(feature_test))

105
45


## Importando PyTorch

In [29]:
import torch
from torch.autograd import Variable 

## Preparando os dados para o PyTorch

Já carregamos os dados nos arrays do NumPy, mas precisamos agrupá-los nos conjuntos de dados PyTorch (nos quais os dados são convertidos em Tensors do PyTorch).

In [30]:
feature_train_t = Variable(torch.FloatTensor(feature_train))
labels_train_t = Variable(torch.FloatTensor(labels_train))
feature_test_t = Variable(torch.FloatTensor(feature_test))
labels_test_t = Variable(torch.FloatTensor(labels_test))

In [31]:
print (feature_train_t[::10])

tensor([[5.0000, 2.0000, 3.5000, 1.0000],
        [6.3000, 2.8000, 5.1000, 1.5000],
        [5.5000, 2.5000, 4.0000, 1.3000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [6.5000, 3.0000, 5.8000, 2.2000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 2.4000, 3.3000, 1.0000],
        [5.0000, 3.3000, 1.4000, 0.2000],
        [5.5000, 2.4000, 3.7000, 1.0000],
        [4.9000, 3.1000, 1.5000, 0.1000]])


In [32]:
print (labels_train_t[::10])

tensor([[0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.]])


##  Neural Network

A camada de entrada recebe 4 valores de entrada (as features da íris: 1. sepal length in cm; 2. sepal width in cm; 3. petal length in cm; 4. petal width in cm ) que manda pra 10 nós da camada oculta e que por sua vez manda pra 3 nós da camada de saída.

In [33]:
class Network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = torch.nn.Linear(4, 10)
        self.output = torch.nn.Linear(10, 3)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.hidden(x)
        x = self.output(x)
        x = self.sigmoid(x)
        return x

model = Network() 
loss_fn = torch.nn.BCELoss() 
optim = torch.optim.SGD(model.parameters(), lr = 0.01)
print(model)

Network(
  (hidden): Linear(in_features=4, out_features=10, bias=True)
  (output): Linear(in_features=10, out_features=3, bias=True)
  (sigmoid): Sigmoid()
)


## Treinar o modelo

Para treinar o modelo, precisamos alimentar repetidamente os valores de treinamento pela rede, usar uma função de perda para calcular a perda, usar um otimizador para retropropagar os ajustes de peso e valor de polarização e validar o modelo usando os dados de teste que retivemos.

Vamos treinar e otimizar o modelo para isso vamos iterar 5000 vezes, registrando as estatísticas de perda e precisão em cada vez.

In [34]:
all_losses = []
for num in range(5000):
    pred = model(feature_train_t) # predict
    loss = loss_fn(pred, labels_train_t) #calculate loss
    all_losses.append(loss.data)
    optim.zero_grad() #zero gradients to not accumulate
    loss.backward() #update weights based on loss
    optim.step() #update optimiser for next iteration

## Testar o Modelo

In [35]:
x_new = [[5.1,3.5,1.4,0.2]]
print ('Nova amostra: {}'.format(x_new[0]))

# Iris-Setosa
x = torch.Tensor(x_new).float()
_, predicted = torch.max(model(x).data, 1)
print('Prediction:',iris_data.target_names[predicted.item()])

Nova amostra: [5.1, 3.5, 1.4, 0.2]
Prediction: setosa


In [36]:
x_new = [[6.4,2.9,4.3,1.3]]
print ('Nova amostra: {}'.format(x_new[0]))

# Iris-versicolor
x = torch.Tensor(x_new).float()
_, predicted = torch.max(model(x).data, 1)
print('Prediction:',iris_data.target_names[predicted.item()])

Nova amostra: [6.4, 2.9, 4.3, 1.3]
Prediction: versicolor


In [37]:
x_new = [[5.8,2.7,5.1,1.9]]
print ('Nova amostra: {}'.format(x_new[0]))

# Iris-virginica
x = torch.Tensor(x_new).float()
_, predicted = torch.max(model(x).data, 1)
print('Prediction:',iris_data.target_names[predicted.item()])

Nova amostra: [5.8, 2.7, 5.1, 1.9]
Prediction: virginica


## Acurácia

In [38]:
from sklearn.metrics import accuracy_score
predicted_values = []

for num in range(len(feature_test_t)):
    predicted_values.append(model(feature_test_t[num]))

In [39]:
score = 0
for num in range(len(predicted_values)):
    if np.argmax(labels_test[num]) == np.argmax(predicted_values[num].data.numpy()):
        score = score + 1

accuracy = float(score / len(predicted_values)) * 100
print ('Accuracy: ' + str(accuracy))

Accuracy: 91.11111111111111
